# Client Code(chatting room)

In [ ]:
import sys
import socket
import threading
from PyQt5.QtWidgets import QApplication, QVBoxLayout, QWidget, QTextEdit, QLineEdit, QPushButton, QLabel, QMenuBar, QStatusBar, QMessageBox, QMainWindow, QFileDialog
from PyQt5.QtGui import QFont
from PyQt5.QtCore import QRect, QMetaObject, QCoreApplication, Qt

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 600)
        self.centralwidget = QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.textEdit = QLineEdit(self.centralwidget)
        self.textEdit.setGeometry(QRect(210, 180, 361, 71))
        self.textEdit.setObjectName("textEdit")
        self.textEdit_2 = QLineEdit(self.centralwidget)
        self.textEdit_2.setGeometry(QRect(210, 300, 361, 71))
        self.textEdit_2.setObjectName("textEdit_2")
        self.label = QLabel(self.centralwidget)
        self.label.setGeometry(QRect(50, 200, 141, 41))        
        font = QFont()
        font.setFamily("微软雅黑")
        font.setPointSize(20)
        font.setBold(True)
        font.setWeight(75)
        self.label.setFont(font)
        self.label.setObjectName("label")
        self.label_2 = QLabel(self.centralwidget)
        self.label_2.setGeometry(QRect(50, 310, 141, 51))
        font = QFont()
        font.setFamily("微软雅黑")
        font.setPointSize(20)
        font.setBold(True)
        font.setWeight(75)
        self.label_2.setFont(font)
        self.label_2.setObjectName("label_2")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QMenuBar(MainWindow)
        self.menubar.setGeometry(QRect(0, 0, 800, 23))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QMetaObject.connectSlotsByName(MainWindow)

        self.textEdit.returnPressed.connect(self.ifReturnPressed)
        self.textEdit_2.returnPressed.connect(self.ifReturnPressed)

    def retranslateUi(self, MainWindow):
        _translate = QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label.setText(_translate("MainWindow", "       IP"))
        self.label_2.setText(_translate("MainWindow", "     端口"))
    
    def ifReturnPressed(self):
        ip = self.textEdit.text()
        port = self.textEdit_2.text()
        try:
            port = int(port) 
            client = ChatClient(ip, port) 
            client.show()
        except ValueError:
            QMessageBox.critical(self, "Error", "Invalid port number")
    


class ChatClient(QWidget):
    def __init__(self, host, port):
        super().__init__()
        self.init_ui()
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.client_socket.connect((host, port))
        except Exception as e:
            QMessageBox.critical(self, "错误", f"连接至服务器失败 {e}")
            self.close()
            return
        threading.Thread(target=self.receive_messages, daemon=True).start()

    def init_ui(self):
        self.setWindowTitle("聊天室")
        layout = QVBoxLayout()
        
        self.chat_display = QTextEdit()
        self.chat_display.setReadOnly(True)
        font = QFont('微软雅黑', 17)
        font.setBold(True)
        self.chat_display.setFont(font)
        layout.addWidget(self.chat_display)

        self.message_input = QLineEdit()
        layout.addWidget(self.message_input)

        self.send_button = QPushButton("Send")
        self.send_button.clicked.connect(self.send_message)
        layout.addWidget(self.send_button)
        self.chat_display.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        # self.send_image_button = QPushButton('Send Image')
        # self.send_image_button.clicked.connect(self.send_image)
        # layout.addWidget(self.send_image_button)


        # self.chat_display.setFocus()
        self.setLayout(layout)
        self.resize(800, 600)
        self.message_input.returnPressed.connect(self.send_message)

    def send_message(self):
        message = self.message_input.text()
        if message:
            self.client_socket.send(message.encode('utf-8'))
            self.message_input.clear()
    
    # def send_image(self):
    #     options = QFileDialog.Options()
    #     file_path, _ = QFileDialog.getOpenFileName(self, "Open Image", "", "Image Files (*.png *.jpg *.bmp)", options=options)
    #     if file_path:
    #         with open(file_path, "rb") as file:
    #             image_data = file.read()
    #         self.client_socket.send(image_data)

    def receive_messages(self):
        initiatedMessageBox = False
        while True:
            try:
                # header = self.client_socket.recv(4)
                # if not header:
                #     break
                # message_length = int.from_bytes(header, byteorder='big')

                message = self.client_socket.recv(1024).decode('utf-8')
                if message == '/receive name' and not initiatedMessageBox:
                    # self.chat_display.clear()
                    initiatedMessageBox = True
                    continue
                if not message:
                    break
                self.chat_display.append(message) 
                self.chat_display.verticalScrollBar().setValue(self.chat_display.verticalScrollBar().maximum())
            except Exception as e:
                print(f"Error receiving message: {e}")
                break
# 😄
    # def closeEvent(self, event):
    #     self.client_socket.close()
    #     event.accept()
if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainwindow = QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(mainwindow)
    mainwindow.show()
    sys.exit(app.exec_())

# client(diceGame)

In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets


class Ui_Form(object):
    def setupUi(self, Form):
        Form.setObjectName("Form")
        Form.resize(659, 467)

        # 创建一个垂直布局管理器
        verticalLayout = QtWidgets.QVBoxLayout(Form)

        # 创建一个水平布局管理器
        horizontalLayout = QtWidgets.QHBoxLayout()

        # 添加按钮到水平布局管理器
        self.pushButton = QtWidgets.QPushButton(Form)
        self.pushButton.setText("玩家1")
        horizontalLayout.addWidget(self.pushButton)

        self.pushButton_2 = QtWidgets.QPushButton(Form)
        self.pushButton_2.setText("玩家2")
        horizontalLayout.addWidget(self.pushButton_2)

        # 将水平布局管理器添加到垂直布局管理器
        verticalLayout.addLayout(horizontalLayout)

        # 添加其他部件到垂直布局管理器
        self.pushButton_3 = QtWidgets.QPushButton(Form)
        self.pushButton_3.setText("玩家3")
        verticalLayout.addWidget(self.pushButton_3)

        self.pushButton_4 = QtWidgets.QPushButton(Form)
        self.pushButton_4.setText("你")
        verticalLayout.addWidget(self.pushButton_4)

        self.frame = QtWidgets.QFrame(Form)
        self.frame.setStyleSheet("QFrame {\n"
                                 "    background-color: #000000; /* 设置为黑色 */\n"
                                 "    height: 2px; /* 调整高度以控制线条粗细 */\n"
                                 "}")
        self.frame.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.frame.setFrameShadow(QtWidgets.QFrame.Raised)
        verticalLayout.addWidget(self.frame)

        self.pushButton_5 = QtWidgets.QPushButton(Form)
        self.pushButton_5.setText("掷骰子")
        verticalLayout.addWidget(self.pushButton_5)

        self.textEdit = QtWidgets.QTextEdit(Form)
        verticalLayout.addWidget(self.textEdit)

        self.textEdit_2 = QtWidgets.QTextEdit(Form)
        verticalLayout.addWidget(self.textEdit_2)

        self.label = QtWidgets.QLabel(Form)
        self.label.setText("系统")
        verticalLayout.addWidget(self.label)

        self.lineEdit = QtWidgets.QLineEdit(Form)
        verticalLayout.addWidget(self.lineEdit)

        self.pushButton_6 = QtWidgets.QPushButton(Form)
        self.pushButton_6.setText("发送")
        verticalLayout.addWidget(self.pushButton_6)

        self.label_2 = QtWidgets.QLabel(Form)
        self.label_2.setText("聊天")
        verticalLayout.addWidget(self.label_2)

        # 设置窗口的主布局
        Form.setLayout(verticalLayout)

        self.retranslateUi(Form)
        QtCore.QMetaObject.connectSlotsByName(Form)

    def retranslateUi(self, Form):
        _translate = QtCore.QCoreApplication.translate
        Form.setWindowTitle(_translate("Form", "Form"))
        self.pushButton.setText(_translate("Form", "玩家1"))
        self.pushButton_2.setText(_translate("Form", "玩家2"))
        self.pushButton_3.setText(_translate("Form", "玩家3"))
        self.pushButton_4.setText(_translate("Form", "你"))
        self.pushButton_5.setText(_translate("Form", "掷骰子"))
        self.label.setText(_translate("Form", "系统"))
        self.pushButton_6.setText(_translate("Form", "发送"))
        self.label_2.setText(_translate("Form", "聊天"))

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Form = QtWidgets.QWidget()
    ui = Ui_Form()
    ui.setupUi(Form)
    Form.show()
    sys.exit(app.exec_())


SystemExit: 0

d:\Env\pyqt5\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
